## Machine learning Model Development

- In this section, we will develop and evaluate machine learning models to predict life expectancy based on various public health metrics, environmental factors, and demographic data. The process will be carried out in a systematic manner, starting from data preparation, model training, and evaluation, followed by model tuning and final evaluation.




In [76]:
# importing libraries and prepared dataset

import pandas as pd
from sklearn.model_selection import train_test_split

data = "/Users/alexandreribeiro/Documents/GitHub/final_project/notebooks/dataset_for_ml.csv"

df = pd.read_csv(data)

df.sample(5)

,population_city,greenspacearea_km2,AQI,adjusted_obesity_rate,adjusted_smoking_rate,adjusted_copd_rate,adjusted_depression_rate,life_expectancy
528,0.643233,-0.871157,0.741016,2.378628,2.641078,2.643804,2.014693,75.0
641,0.504034,-0.223973,0.272914,1.023043,0.891780,0.895913,1.567798,79.0
2938,-0.803046,0.532011,0.419384,0.472279,0.111061,0.156046,0.429652,78.4
1264,-0.107850,-1.111993,-0.072002,-0.081826,0.139599,0.166644,-0.026337,75.3
559,0.598268,0.252743,0.420736,2.568876,3.236579,0.597030,1.130262,75.1


In [77]:
# Applying the logical inverse (multiplying by -1) to all features except the target ('adjusted_life_expectancy')
features_to_inverse = [
    'adjusted_obesity_rate', 
    'adjusted_smoking_rate', 
    'adjusted_copd_rate', 
    'adjusted_depression_rate'
]

# Creating the inverse for each of these features
for feature in features_to_inverse:
    df[f'logical_inverse_{feature}'] = -df[feature]

# Display the first few rows of the updated dataframe to confirm
df.head()

,population_city,greenspacearea_km2,AQI,adjusted_obesity_rate,adjusted_smoking_rate,adjusted_copd_rate,adjusted_depression_rate,life_expectancy,logical_inverse_adjusted_obesity_rate,logical_inverse_adjusted_smoking_rate,logical_inverse_adjusted_copd_rate,logical_inverse_adjusted_depression_rate
0,5.580401,-0.012540,0.014227,1.682852,1.389244,1.628696,1.211327,77.5,-1.682852,-1.389244,-1.628696,-1.211327
1,5.553693,0.516264,-0.078162,1.117487,0.624299,0.587606,0.611860,76.5,-1.117487,-0.624299,-0.587606,-0.611860
2,5.520049,0.080073,-0.946613,1.654772,1.364492,1.602239,1.187964,77.5,-1.654772,-1.364492,-1.602239,-1.187964
3,5.394723,-0.529781,1.806562,0.032259,-0.171918,-0.243810,0.002308,79.0,-0.032259,0.171918,0.243810,-0.002308
4,5.314361,-0.271656,2.521032,0.012965,-0.187696,-0.258318,-0.015646,79.0,-0.012965,0.187696,0.258318,0.015646


#### Defining features and target variable

- The first step in developing a machine learning model is to define the features and target variable. In this case, the target variable is the life expectancy, and the features are the various public health metrics, environmental factors, and demographic data. We will use the following features to predict life expectancy.

In [78]:
# Selected features

selected_features = ['population_city', 'greenspacearea_km2', 'AQI', 'adjusted_obesity_rate',
       'adjusted_smoking_rate', 'adjusted_copd_rate',
       'adjusted_depression_rate', 'life_expectancy',
       'logical_inverse_adjusted_obesity_rate',
       'logical_inverse_adjusted_smoking_rate',
       'logical_inverse_adjusted_copd_rate',
       'logical_inverse_adjusted_depression_rate']

# Define the target variable (y) and the feature set (X)

y = df['life_expectancy']  # Target variable
X = df[selected_features]  # Using the selected features

# Display the shapes of X and y

X.shape, y.shape

((3238, 12), (3238,))

#### Spliting the data into training and testing sets

- Split the dataset into training and testing sets to train the model on one portion and test it on another.

- test_size=0.2: This means that 20% of the data will be used for testing, and the remaining 80% will be used for training.
- random_state=42: Setting a random state ensures that the split is reproducible.

In [79]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (2590, 12)
X_test shape: (648, 12)
y_train shape: (2590,)
y_test shape: (648,)


#### Model Selection:

- We will start with a few common regression models since you’re predicting a continuous target variable (adjusted life expectancy).

We’ll try these models:

- Linear Regression
- Decision Tree Regressor
- Random Forest Regressor
- Gradient Boosting Regressor
- Support Vector Regressor (SVR)

- After selecting the models, we’ll compare their performance to choose the best one.

In [80]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define the models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "SVR": SVR()
}

# Dictionary to store the results
results = {}

# Train and evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Store the results
    results[model_name] = {
        "MAE": mae,
        "MSE": mse,
        "R²": r2
    }

    print(f"{model_name} results:")
    print(f"  - MAE: {mae:.3f}")
    print(f"  - MSE: {mse:.3f}")
    print(f"  - R²: {r2:.3f}\n")

# Convert the results to a DataFrame for easier comparison
import pandas as pd
results_df = pd.DataFrame(results).T
print(results_df)

Linear Regression results:
  - MAE: 0.000
  - MSE: 0.000
  - R²: 1.000

Decision Tree results:
  - MAE: 0.000
  - MSE: 0.000
  - R²: 1.000

Random Forest results:
  - MAE: 0.000
  - MSE: 0.000
  - R²: 1.000

Gradient Boosting results:
  - MAE: 0.000
  - MSE: 0.000
  - R²: 1.000

SVR results:
  - MAE: 0.141
  - MSE: 0.030
  - R²: 0.985

                            MAE           MSE        R²
Linear Regression  6.469448e-15  9.692276e-29  1.000000
Decision Tree      1.523719e-13  5.803211e-26  1.000000
Random Forest      2.407407e-04  3.891975e-06  0.999998
Gradient Boosting  6.860396e-05  2.209917e-08  1.000000
SVR                1.409372e-01  3.025712e-02  0.985210


In [81]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

# Define the models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "SVR": SVR()
}

# Dictionary to store the results
results = []

# Train and evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Store the results in the list
    results.append({
        "Model": model_name,
        "MAE": mae,
        "MSE": mse,
        "R²": r2
    })

# Convert the results to a DataFrame for easier comparison
results_df = pd.DataFrame(results)
results_df

,Model,MAE,MSE,R²
0,Linear Regression,6.469448e-15,9.692276e-29,1.000000
1,Decision Tree,1.500035e-13,5.771486e-26,1.000000
2,Random Forest,1.635802e-04,2.700617e-06,0.999999
3,Gradient Boosting,6.860396e-05,2.209917e-08,1.000000
4,SVR,1.409372e-01,3.025712e-02,0.985210


#### Cross-validation:

- We will use cross-validation to evaluate the performance of the models. Cross-validation is a technique that splits the data into multiple subsets (folds) and trains the model on different combinations of these subsets. This helps to get a more accurate estimate of the model’s performance.

In [82]:
from sklearn.model_selection import cross_val_score
import numpy as np

# Define the model
model = RandomForestRegressor()  # You can choose the model you prefer

# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')

# Output the cross-validation results
print("Cross-Validation R² Scores: ", cv_scores)
print("Mean R²: ", np.mean(cv_scores))
print("Standard Deviation of R²: ", np.std(cv_scores))

Cross-Validation R² Scores:  [0.99999847 0.99999998 0.99990293 0.99993479 0.99999342]
Mean R²:  0.9999659177262752
Standard Deviation of R²:  3.9781460324212305e-05


#### Results:

- Cross-Validation R² Scores: The individual R² scores across the 5 folds are all very high, indicating that the model is consistently performing well on different subsets of the data.

- Mean R²: The average R² score across the 5 folds is approximately 0.984, which suggests that the model explains about 98.4% of the variance in the target variable.

- Standard Deviation of R²: The standard deviation is very low (0.0021), indicating that the model’s performance is consistent across different data splits. This is a good sign as it shows that the model is not overly dependent on any particular subset of the data.


In [83]:
# Lets check the remaining models

# List of models to evaluate

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "SVR": SVR()
}

# Dictionary to store the cross-validation results
cv_results = {}

# Perform cross-validation for each model
for model_name, model in models.items():
    # Perform cross-validation with 5 folds
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    
    # Store the mean and standard deviation of R² scores
    cv_results[model_name] = {
        "Mean R²": cv_scores.mean(),
        "Std Dev R²": cv_scores.std()
    }

# Convert the results to a DataFrame for easier comparison
cv_results_df = pd.DataFrame(cv_results).T
cv_results_df

,Mean R²,Std Dev R²
Linear Regression,1.000000,0.000000e+00
Decision Tree,0.999947,1.065175e-04
Random Forest,0.999954,5.399618e-05
Gradient Boosting,1.000000,2.364946e-08
SVR,0.959249,3.545789e-03


#### Using Hyperparameter Tuning to try to improve the model performance

- Using GridSearchCV to find the best hyperparameters for the model. GridSearchCV is a technique that searches for the best combination of hyperparameters by evaluating the model’s performance on different combinations of hyperparameters.

In [84]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Random Forest Regressor
rf = RandomForestRegressor()

# Initialize GridSearchCV with 5-fold cross-validation
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, 
                              cv=5, scoring='r2', n_jobs=-1, verbose=2)

# Fit the GridSearchCV on the training data
grid_search_rf.fit(X_train, y_train)

# Best parameters and the corresponding score
best_params_rf = grid_search_rf.best_params_
best_score_rf = grid_search_rf.best_score_

print(f"Best Parameters for Random Forest: {best_params_rf}")
print(f"Best R² Score for Random Forest: {best_score_rf:.4f}")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=

/opt/anaconda3/envs/env_test/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
99 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/env_test/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/envs/env_test/lib/python3.11/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/opt/anaconda3/envs/env_test/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_c

Best Parameters for Random Forest: {'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best R² Score for Random Forest: 0.9911


In [85]:
# Get the best model from GridSearchCV
best_rf = grid_search_rf.best_estimator_

# Make predictions on the test set
y_pred_rf = best_rf.predict(X_test)

# Evaluate the performance
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest - Tuned Model Performance:")
print(f"  - MAE: {mae_rf:.3f}")
print(f"  - MSE: {mse_rf:.3f}")
print(f"  - R²: {r2_rf:.3f}")

Random Forest - Tuned Model Performance:
  - MAE: 0.068
  - MSE: 0.016
  - R²: 0.992


#### Overfitting check

- Compare the performance of the model on the training data versus the test data. A common method is to look at the difference between training and validation scores, especially during cross-validation.

In [86]:
# Assuming you've already done hyperparameter tuning and have the best model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Train the best model with the entire training set
best_rf_model = RandomForestRegressor(
    max_depth=20, 
    max_features='log2', 
    min_samples_leaf=1, 
    min_samples_split=2, 
    n_estimators=300,
    random_state=42
)

best_rf_model.fit(X_train, y_train)

# Evaluate the model on the training data
y_train_pred = best_rf_model.predict(X_train)

# Calculate metrics on training data
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print("Random Forest - Training Data Performance:")
print(f"  - MAE: {train_mae:.3f}")
print(f"  - MSE: {train_mse:.3f}")
print(f"  - R²: {train_r2:.3f}\n")

Random Forest - Training Data Performance:
  - MAE: 0.025
  - MSE: 0.002
  - R²: 0.999



In [87]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define the best model with the optimal hyperparameters
best_rf_model = RandomForestRegressor(
    max_depth=20, 
    max_features='log2', 
    min_samples_leaf=1, 
    min_samples_split=2, 
    n_estimators=300,
    random_state=42
)

# Train the model on the entire training set
best_rf_model.fit(X_train, y_train)

# Evaluate the model on the training data
y_train_pred = best_rf_model.predict(X_train)

# Calculate metrics on training data
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print("Random Forest - Training Data Performance:")
print(f"  - MAE: {train_mae:.3f}")
print(f"  - MSE: {train_mse:.3f}")
print(f"  - R²: {train_r2:.3f}\n")

# Evaluate the model on the test data
y_test_pred = best_rf_model.predict(X_test)

# Calculate metrics on test data
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Random Forest - Test Data Performance:")
print(f"  - MAE: {test_mae:.3f}")
print(f"  - MSE: {test_mse:.3f}")
print(f"  - R²: {test_r2:.3f}\n")

Random Forest - Training Data Performance:
  - MAE: 0.025
  - MSE: 0.002
  - R²: 0.999

Random Forest - Test Data Performance:
  - MAE: 0.066
  - MSE: 0.014
  - R²: 0.993



#### Conclusion:

- A difference of 0.02 in R² is not a strong indication of overfitting. It’s more likely a sign that the model is performing well. Overfitting concerns arise more when there are larger discrepancies between training and test performance. In this case the model seems to be well-tuned, and the small difference are likely acceptable. 

#### Trying XGBoost Regressor

- XGBoost is a powerful and efficient implementation of the gradient boosting algorithm. It is known for its speed and performance, making it a popular choice for machine learning tasks. We will use the XGBoost Regressor to predict life expectancy and evaluate its performance.

In [88]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# Initialize XGBoost model

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Train the model

xgb_model.fit(X_train, y_train)

# Make predictions on the test set

y_pred = xgb_model.predict(X_test)

# Evaluate the model

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display the evaluation metrics in a dataframe

results = pd.DataFrame(data=[mae, mse, r2], index=['MAE', 'MSE', 'R²'], columns=['XGBoost'])

results



,XGBoost
MAE,2.816989e-05
MSE,1.407422e-08
R²,1.000000e+00


#### Hyperparameter Tuning for XGBoost Regressor

In [89]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print("Best Parameters for XGBoost:", grid_search.best_params_)
print("Best R² Score for XGBoost:", grid_search.best_score_)

# Evaluate the best model on the test set
best_xgb_model = grid_search.best_estimator_
y_pred_best = best_xgb_model.predict(X_test)

# Calculate metrics on test data
best_mae = mean_absolute_error(y_test, y_pred_best)
best_mse = mean_squared_error(y_test, y_pred_best)
best_r2 = r2_score(y_test, y_pred_best)

# Display the evaluation metrics in a dataframe

results['XGBoost Tuned'] = [best_mae, best_mse, best_r2]

results

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END

,XGBoost,XGBoost Tuned
MAE,2.816989e-05,0.002526
MSE,1.407422e-08,0.000036
R²,1.000000e+00,0.999983


##### Results:

- MAE (Mean Absolute Error) decreased from 0.0667 to 0.0593, indicating that the average error between the predicted and actual values has reduced.

- MSE (Mean Squared Error) also decreased from 0.0140 to 0.0121, which means that the model’s overall error has been reduced, especially penalizing larger errors.

- R² (Coefficient of Determination) increased slightly from 0.9873 to 0.9890, showing that the model now explains slightly more variance in the data.

#### Checking for overfitting on XGBoost Regressor

In [90]:
import xgboost as xgb

# Best parameters from your hyperparameter tuning
best_params_xgb = {
    'colsample_bytree': 0.9,
    'learning_rate': 0.1,
    'max_depth': 7,
    'n_estimators': 300,
    'subsample': 0.9
}

# Initialize and train the XGBoost model with the best parameters
tuned_xgb_model = xgb.XGBRegressor(**best_params_xgb)
tuned_xgb_model.fit(X_train, y_train)

# Now, evaluate the tuned XGBoost model on both the training and test data
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluate on the training data
y_train_pred_xgb = tuned_xgb_model.predict(X_train)
train_mae_xgb = mean_absolute_error(y_train, y_train_pred_xgb)
train_mse_xgb = mean_squared_error(y_train, y_train_pred_xgb)
train_r2_xgb = r2_score(y_train, y_train_pred_xgb)

print("Tuned XGBoost - Training Data Performance:")
print(f"  - MAE: {train_mae_xgb:.3f}")
print(f"  - MSE: {train_mse_xgb:.3f}")
print(f"  - R²: {train_r2_xgb:.3f}\n")

# Evaluate on the test data
y_test_pred_xgb = tuned_xgb_model.predict(X_test)
test_mae_xgb = mean_absolute_error(y_test, y_test_pred_xgb)
test_mse_xgb = mean_squared_error(y_test, y_test_pred_xgb)
test_r2_xgb = r2_score(y_test, y_test_pred_xgb)

print("Tuned XGBoost - Test Data Performance:")
print(f"  - MAE: {test_mae_xgb:.3f}")
print(f"  - MSE: {test_mse_xgb:.3f}")
print(f"  - R²: {test_r2_xgb:.3f}")

Tuned XGBoost - Training Data Performance:
  - MAE: 0.001
  - MSE: 0.000
  - R²: 1.000

Tuned XGBoost - Test Data Performance:
  - MAE: 0.013
  - MSE: 0.001
  - R²: 1.000


#### Overfitting further investigation 

- Although both models (XGBoost and Random Forest) have very good R2 scores, the XGBoost has a perfect R2 score on the training data, which is a sign of overfitting. The Random Forest model has a more balanced performance on the training and test data, which is a good sign. 

#### Applying Regularization to XGBoost Regressor

- Applying regularization in XGBoost, these penalties are added to the objective function that the model is trying to minimize. The objective function in XGBoost typically consists of two parts:

1.	Loss function: Measures the difference between the predicted and actual values.
2.	Regularization term: Penalizes the complexity of the model (through alpha and lambda).

In [91]:
# Set up the model and hyperparameters

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Initialize the XGBoost model

xgb_model = xgb.XGBRegressor(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=7,
    subsample=0.9,
    colsample_bytree=0.9
)

# Define the hyperparameter grid

param_grid = {
    'reg_alpha': [0, 0.1, 0.5, 1, 5],  # L1 regularization
    'reg_lambda': [0.5, 1, 2, 5, 10]   # L2 regularization
}

##### Explanation:

- reg_alpha: We are testing values ranging from no regularization (0) to stronger regularization (5).
- reg_lambda: Similar to reg_alpha, we’re testing a range of values to see how different levels of L2 regularization affect the model.

In [92]:
# Set up the GridSearchCV with cross-validation

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='r2',  # Use R² as the scoring metric
    n_jobs=-1,  # Use all available cores
    verbose=1
)

# Fit the grid search to the training data

grid_search.fit(X_train, y_train)

# Extract the best parameters and score

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters for XGBoost:", best_params)
print("Best R² Score for XGBoost:", best_score)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best Parameters for XGBoost: {'reg_alpha': 0, 'reg_lambda': 10}
Best R² Score for XGBoost: 0.9996214554855932


##### Explanation:

- GridSearchCV: We use it to systematically work through multiple combinations of parameter values, cross-validating each combination to determine which one gives the best performance.
- cv=5: We are using 5-fold cross-validation to evaluate the model’s performance. The dataset is split into 5 parts, and the model is trained on 4 parts and tested on the remaining part. This is repeated 5 times.
- scoring=‘r2’: We are using R² as the scoring metric to evaluate the model’s performance.


In [93]:
# Retrain the XGBoost model with the best parameters

tuned_xgb_model = xgb.XGBRegressor(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=7,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda']
)

# Fit the model on the training data

tuned_xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

##### Explanation:

- We use the best reg_alpha and reg_lambda values found during the grid search to train the final model.

In [94]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluate the tuned model on both training and test data
y_train_pred = tuned_xgb_model.predict(X_train)
y_test_pred = tuned_xgb_model.predict(X_test)

# Calculate metrics on training data
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Calculate metrics on test data
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    "Dataset": ["Training", "Test"],
    "MAE": [train_mae, test_mae],
    "MSE": [train_mse, test_mse],
    "R²": [train_r2, test_r2]
})

# Display the DataFrame
print(results_df)

    Dataset       MAE       MSE        R²
0  Training  0.002428  0.000016  0.999992
1      Test  0.013537  0.000791  0.999613


#### Stacking Models for Better Performance

- Stacking is an ensemble learning technique that combines multiple models to improve performance. It works by training multiple base models on the same data and then combining their predictions using a meta-model. The meta-model learns how to best combine the base models to make the final prediction.

**Selected Models:**  *Random Forest, SVR, and XGBoost*

In [95]:
# Hyperparameter tuning for SVR to find the best parameters

from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for SVR
svr_param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.5, 1],
    'kernel': ['linear', 'rbf']
}

# Initialize the SVR model
svr_model = SVR()

# Set up GridSearchCV
svr_grid_search = GridSearchCV(estimator=svr_model, param_grid=svr_param_grid, cv=5, scoring='r2', n_jobs=-1)

# Perform the grid search
svr_grid_search.fit(X_train, y_train)

# Get the best parameters
best_svr_params = svr_grid_search.best_params_
print("Best Parameters for SVR:", best_svr_params)
print("Best R² Score for SVR:", svr_grid_search.best_score_)

Best Parameters for SVR: {'C': 100, 'epsilon': 0.01, 'kernel': 'rbf'}
Best R² Score for SVR: 0.999990723216866


#### Now that we have the best params to each model, we can stack them together to create a final model.

**Base Models with Tuned Parameters:**

- Random Forest with max_depth=20, max_features='log2', min_samples_leaf=1, min_samples_split=2, n_estimators=300.
- XGBoost with colsample_bytree=0.9, learning_rate=0.1, max_depth=7, n_estimators=300, subsample=0.9.
- SVR with C=10, epsilon=0.01, kernel='rbf'.

In [96]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Best parameters for each model

best_rf_params = {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
best_xgb_params = {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'subsample': 0.9}
best_svr_params = {'C': 10, 'epsilon': 0.01, 'kernel': 'rbf'}

# Define the base models with the tuned parameters

base_models = [
    ("Random Forest", RandomForestRegressor(**best_rf_params)),  
    ("XGBoost", XGBRegressor(**best_xgb_params)),                
    ("SVR", SVR(**best_svr_params))  # Use the best params for SVR
]

# Initialize the meta-model (Linear Regression)

meta_model = LinearRegression()

# Cross-validation predictions for base models

base_predictions = np.zeros((X_train.shape[0], len(base_models)))

for i, (name, model) in enumerate(base_models):
    base_predictions[:, i] = cross_val_predict(model, X_train, y_train, cv=5)

# Train the meta-model on the predictions of base models

meta_model.fit(base_predictions, y_train)

# Predict on test data using the base models

test_base_predictions = np.zeros((X_test.shape[0], len(base_models)))

for i, (name, model) in enumerate(base_models):
    model.fit(X_train, y_train)  # Train on full training data
    test_base_predictions[:, i] = model.predict(X_test)

# Meta-model predictions

final_predictions = meta_model.predict(test_base_predictions)

# Evaluate the stacked model

train_mae = mean_absolute_error(y_train, meta_model.predict(base_predictions))
train_mse = mean_squared_error(y_train, meta_model.predict(base_predictions))
train_r2 = r2_score(y_train, meta_model.predict(base_predictions))

test_mae = mean_absolute_error(y_test, final_predictions)
test_mse = mean_squared_error(y_test, final_predictions)
test_r2 = r2_score(y_test, final_predictions)

# Store the results in a DataFrame

results_df = pd.DataFrame({
    "Dataset": ["Training", "Test"],
    "MAE": [train_mae, test_mae],
    "MSE": [train_mse, test_mse],
    "R²": [train_r2, test_r2]
})

print(results_df)

    Dataset       MAE       MSE        R²
0  Training  0.003227  0.000017  0.999992
1      Test  0.003328  0.000018  0.999991


#### Result summary:

- This is our best result yet, with a minimal difference between training and test scores, indicating that the model is well-tuned and not overfitting. The R² score of 0.9895 is very high, indicating that the model explains 98.95% of the variance in the target variable. The MAE and MSE are also very low, indicating that the model’s predictions are close to the actual values.

#### Training the best model on the full dataset

In [99]:
df = df.copy()

# Define the target and features

X = df.drop(columns=['life_expectancy'])
y = df['life_expectancy']

# Standardizing the features

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train the base models

best_rf_model = RandomForestRegressor(**best_rf_params)
best_xgb_model = XGBRegressor(**best_xgb_params)
best_svr_model = SVR(**best_svr_params)

# Fit each base model on the full dataset

best_rf_model.fit(X_scaled, y)
best_xgb_model.fit(X_scaled, y)
best_svr_model.fit(X_scaled, y)

# Stack predictions from the base models

base_predictions = np.column_stack([
    best_rf_model.predict(X_scaled),
    best_xgb_model.predict(X_scaled),
    best_svr_model.predict(X_scaled)
])

# Train the meta-model (Linear Regression) on the base models' predictions

meta_model = LinearRegression()
meta_model.fit(base_predictions, y)

# Save the models and scaler

with open('stacked_model.pkl', 'wb') as file:
    pickle.dump({'base_models': [best_rf_model, best_xgb_model, best_svr_model], 'meta_model': meta_model}, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# Evaluate the model using cross-validation

meta_predictions = cross_val_predict(meta_model, base_predictions, y, cv=5)

# Calculate performance metrics

mae = mean_absolute_error(y, meta_predictions)
mse = mean_squared_error(y, meta_predictions)
r2 = r2_score(y, meta_predictions)

# Store the results in a DataFrame for easier comparison

results_df = pd.DataFrame({
    "Metric": ["MAE", "MSE", "R²"],
    "Value": [mae, mse, r2]
})

print(results_df)

  Metric     Value
0    MAE  0.045389
1    MSE  0.004057
2     R²  0.997992
